# Efficiency - Delete multiple points

| comparable algo list |
| - |
| delta |
| Monte Carlo |

In [ ]:
import sys
sys.path.append("../..")

from sklearn import svm
import numpy as np
from copy import deepcopy

import src

from src.data_utils import (
    load_npy, save_npy, preprocess_data, variance, normalize, generate_union_description
)

In [ ]:
model = svm.SVC(decision_function_shape='ovo')

size_choices = [10, 100, 1000, 10000]
m_choices = [10, 15, 18, 20, 22, 25, 28, 30]
stone_m = 100

# for 200 points
size_choices = [200]
m_choices = [1, 2, 3, 5, 8]
mc_stone = 30

for size in size_choices:
    x_train, y_train, x_test, y_test, \
    columns_name = preprocess_data(
        't_train_' + str(size) + 'p.csv',
        't_test_' + str(size) + 'p.csv')

    # process data
    x_train_ = np.append(x_train, x_test[:2], axis=0)
    y_train_ = np.append(y_train, y_test[:2])
    x_test_ = x_test[2:, :]
    y_test_ = y_test[2:]

    union_d = generate_union_description(len(x_train), 10)
    union_d_ = deepcopy(union_d)
    for idx in [0, 1]:
        for _ in union_d_:
            if idx in _:
                _.remove(idx)
                break
    union_d_delete_last = deepcopy(union_d)
    to_add = 0
    for i in len(union_d_delete_last): 
        if len(x_train) in union_d_delete_last[i]:
            to_add = i
            union_d_delete_last[i].remove(len(x_train))
            break 

    # get a init ov
    init_sv = src.mc_owen(x_train_, y_train_, x_test_, y_test_, model, union_d_, stone_m * len(y_train_), proc_num=20)
    save_npy('dm_init_mc_plus_size' + str(size) + '.npy', init_sv)

    # set a stone
    mc_plus = src.mc_owen(x_train, y_train, x_test_, y_test_, model, union_d, stone_m * len(y_train), proc_num=20)
    save_npy('dm_mc_plus_size' + str(size) + '.npy', mc_plus)

    # mc
    mc = src.mc_owen(x_train, y_train, x_test_, y_test_, model, union_d, mc_stone * len(y_train), proc_num=20)
    save_npy('dm_mc_size'+ str(size) +'m' + str(mc_stone) + '.npy', mc)

    # # walk the m choices
    for m in m_choices:
       # delta
        delta_owen = src.DeltaOwen(x_train_, y_train_, x_test_, y_test_, model,
                                        init_sv, union_d_)
        m_delt_ov = delta_owen.del_single_point(len(y_train_) - 1, m=(len(y_train_)-1)*m, proc_num=20)

        n_delta_owen = src.DeltaOwen(x_train_[:-1,:], y_train_[:-1], x_test_,
                                        y_test_, model, m_delt_ov, union_d_delete_last)
        delt_ov = n_delta_owen.del_single_point(len(y_train_) - 2, (len(y_train_)-2)*m, proc_num=20)
        save_npy('dm_delta_size' + str(size) +'m' + str(m) + '.npy', delt_ov)